In [1]:
import pandas as pd
import numpy as np
import torch
import json
import os
from Network.deepnetwork2 import ImitationNetwork
from Memory.dataset import *
from torch.utils.data import DataLoader
from Utils.utils import *
from sklearn.metrics import precision_score, recall_score
import torch.nn as nn

In [ ]:
len(decode_state(a))

In [ ]:
a = {'player_id': 1, 'player_life': 39, 'player_gold': 0, 'opponent_life': 39, 'time': 59.839996337890625, 'player_board_card_info': [{'id': 23, 'status': 1, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 50, 'status': 2, 'damage': 2, 'life': 1}, {'id': 50, 'status': 2, 'damage': 
2, 'life': 1}, {'id': 31, 'status': 2, 'damage': 1, 'life': 1}, {'id': 20, 'status': 0, 'damage': 3, 'life': 3}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'opponent_board_card_info': [{'id': 23, 'status': 0, 'damage': 1, 'life': 2}, {'id': 31, 'status': 0, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 34, 'status': 0, 'damage': 2, 'life': 1}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'player_hand_card_id': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'player_remaining_cards_on_deck': [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], 'opponent_remaining_cards': [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'player_used_cards': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'opponent_used_cards': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],'action':21}

In [8]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('TensorBoard/ImitationAgent_AllActions_NewData_OldDecode/')

In [9]:
dataset = Feeder3('Log/Train/')

In [10]:
dataloader = DataLoader(dataset,batch_size = 256, num_workers = 10)

In [11]:
def accuracy(pred, label):
    pred = torch.argmax(pred, dim=1).long()
#     print(pred)
    label = torch.argmax(label,dim=1).long()
    acc = torch.mean((pred == label).float())
    pred = pred.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    p = precision_score(label, pred,average='micro')
    r = recall_score(label, pred,average='micro')
    return p,r,acc 

class AverageMeter(object):

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
for i, (data,reward,label) in enumerate(dataloader):
    print(data.shape)
    break

torch.Size([256, 516])


In [13]:
model = ImitationNetwork(0.1,6,'Test',516,'Test')
model.to(torch.device('cuda:2'))

ImitationNetwork(
  (dense1): Linear(in_features=516, out_features=2048, bias=True)
  (dense2): Linear(in_features=2048, out_features=1024, bias=True)
  (dense3): Linear(in_features=1024, out_features=512, bias=True)
  (dense4): Linear(in_features=512, out_features=256, bias=True)
  (dense5): Linear(in_features=256, out_features=128, bias=True)
  (dense6): Linear(in_features=128, out_features=64, bias=True)
  (dense7): Linear(in_features=64, out_features=32, bias=True)
  (dense8): Linear(in_features=32, out_features=6, bias=True)
  (loss): CrossEntropyLoss()
)

In [14]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:2'))
losses = 0.0
accs  = 0.0
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, 
                          weight_decay=1e-4) 
checkpoint = torch.load('Models/1stImitationAgent_NewData_Oldecode.ckpt')
model.load_state_dict(checkpoint)
for epoch in range(20):
    for i, (data,reward,label) in enumerate(dataloader):
        data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:2')),(data,reward,label))

        pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
        loss = crit(pred,torch.nn.functional.softmax(reward_))
        p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

        opt.zero_grad()
        loss.backward()
        opt.step()

#         losses.update(loss.item(),530)
#         accs.update(acc.item(),530)
        losses += loss.item()
        accs += acc.item()
        precisions.update(p, 530)
        recalls.update(r,530)

        if i % 5 == 0:
            print('Epoch:[{0}][{1}/{2}]\t'
#                   'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
#                   'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
                  'Loss {losses:.3f} \t'
                  'Accuracy {accs:.3f}\t'
                  'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
                  'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                        epoch,i,len(dataloader) ,losses=loss, accs=acc, 
                        precisions=precisions, recalls=recalls))
        if i % 10 == 0:
            torch.save(model.state_dict(), 'Models/1stImitationAgent_NewData_Oldecode.ckpt')
            # ...log the running loss
            writer.add_scalar('training loss',
                            losses / 10,
                            epoch * len(dataloader) + i + 5457)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_scalar('accuracy',
                            accs / 10,
                            global_step=epoch * len(dataloader) + i + 5457)
            losses = 0.0
            accs = 0.0
    #         print('Epoch:[{0}][{1}/{2}]\t'
    #               'Loss {losses:.3f} \t'
    #               'Accuracy {accs:.3f}\t'
    #               'Precison {precisions:.3f}\t'
    #               'Recall {recalls:.3f}'.format(
    #                     epoch, losses=loss, accs=acc, 
    #                     precisions=p, recalls=r))
    

/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[0][0/424]	Loss 0.263 	Accuracy 0.598	Precison 0.598 (0.598)	Recall 0.598 (0.598)
Epoch:[0][5/424]	Loss 0.294 	Accuracy 0.625	Precison 0.625 (0.599)	Recall 0.625 (0.599)
Epoch:[0][10/424]	Loss 0.362 	Accuracy 0.531	Precison 0.531 (0.580)	Recall 0.531 (0.580)
Epoch:[0][15/424]	Loss 0.317 	Accuracy 0.633	Precison 0.633 (0.578)	Recall 0.633 (0.578)
Epoch:[0][20/424]	Loss 0.318 	Accuracy 0.508	Precison 0.508 (0.572)	Recall 0.508 (0.572)
Epoch:[0][25/424]	Loss 0.343 	Accuracy 0.543	Precison 0.543 (0.565)	Recall 0.543 (0.565)
Epoch:[0][30/424]	Loss 0.330 	Accuracy 0.582	Precison 0.582 (0.566)	Recall 0.582 (0.566)
Epoch:[0][35/424]	Loss 0.361 	Accuracy 0.527	Precison 0.527 (0.572)	Recall 0.527 (0.572)
Epoch:[0][40/424]	Loss 0.349 	Accuracy 0.535	Precison 0.535 (0.570)	Recall 0.535 (0.570)
Epoch:[0][45/424]	Loss 0.229 	Accuracy 0.758	Precison 0.758 (0.580)	Recall 0.758 (0.580)
Epoch:[0][50/424]	Loss 0.224 	Accuracy 0.750	Precison 0.750 (0.598)	Recall 0.750 (0.598)
Epoch:[0][55/424]	Loss 

Epoch:[1][35/424]	Loss 0.364 	Accuracy 0.543	Precison 0.543 (0.573)	Recall 0.543 (0.573)
Epoch:[1][40/424]	Loss 0.351 	Accuracy 0.500	Precison 0.500 (0.573)	Recall 0.500 (0.573)
Epoch:[1][45/424]	Loss 0.223 	Accuracy 0.758	Precison 0.758 (0.574)	Recall 0.758 (0.574)
Epoch:[1][50/424]	Loss 0.235 	Accuracy 0.750	Precison 0.750 (0.576)	Recall 0.750 (0.576)
Epoch:[1][55/424]	Loss 0.385 	Accuracy 0.746	Precison 0.746 (0.577)	Recall 0.746 (0.577)
Epoch:[1][60/424]	Loss 0.382 	Accuracy 0.426	Precison 0.426 (0.577)	Recall 0.426 (0.577)
Epoch:[1][65/424]	Loss 0.343 	Accuracy 0.355	Precison 0.355 (0.577)	Recall 0.355 (0.577)
Epoch:[1][70/424]	Loss 0.343 	Accuracy 0.500	Precison 0.500 (0.576)	Recall 0.500 (0.576)
Epoch:[1][75/424]	Loss 0.313 	Accuracy 0.668	Precison 0.668 (0.576)	Recall 0.668 (0.576)
Epoch:[1][80/424]	Loss 0.380 	Accuracy 0.379	Precison 0.379 (0.576)	Recall 0.379 (0.576)
Epoch:[1][85/424]	Loss 0.352 	Accuracy 0.480	Precison 0.480 (0.576)	Recall 0.480 (0.576)
Epoch:[1][90/424]	Los

Epoch:[2][70/424]	Loss 0.344 	Accuracy 0.664	Precison 0.664 (0.574)	Recall 0.664 (0.574)
Epoch:[2][75/424]	Loss 0.322 	Accuracy 0.613	Precison 0.613 (0.574)	Recall 0.613 (0.574)
Epoch:[2][80/424]	Loss 0.390 	Accuracy 0.410	Precison 0.410 (0.574)	Recall 0.410 (0.574)
Epoch:[2][85/424]	Loss 0.352 	Accuracy 0.516	Precison 0.516 (0.574)	Recall 0.516 (0.574)
Epoch:[2][90/424]	Loss 0.389 	Accuracy 0.590	Precison 0.590 (0.574)	Recall 0.590 (0.574)
Epoch:[2][95/424]	Loss 0.333 	Accuracy 0.617	Precison 0.617 (0.574)	Recall 0.617 (0.574)
Epoch:[2][100/424]	Loss 0.359 	Accuracy 0.707	Precison 0.707 (0.574)	Recall 0.707 (0.574)
Epoch:[2][105/424]	Loss 0.394 	Accuracy 0.559	Precison 0.559 (0.574)	Recall 0.559 (0.574)
Epoch:[2][110/424]	Loss 0.257 	Accuracy 0.742	Precison 0.742 (0.574)	Recall 0.742 (0.574)
Epoch:[2][115/424]	Loss 0.304 	Accuracy 0.594	Precison 0.594 (0.575)	Recall 0.594 (0.575)
Epoch:[2][120/424]	Loss 0.332 	Accuracy 0.727	Precison 0.727 (0.575)	Recall 0.727 (0.575)
Epoch:[2][125/42

Epoch:[3][105/424]	Loss 0.402 	Accuracy 0.504	Precison 0.504 (0.574)	Recall 0.504 (0.574)
Epoch:[3][110/424]	Loss 0.268 	Accuracy 0.680	Precison 0.680 (0.574)	Recall 0.680 (0.574)
Epoch:[3][115/424]	Loss 0.309 	Accuracy 0.566	Precison 0.566 (0.574)	Recall 0.566 (0.574)
Epoch:[3][120/424]	Loss 0.331 	Accuracy 0.676	Precison 0.676 (0.574)	Recall 0.676 (0.574)
Epoch:[3][125/424]	Loss 0.337 	Accuracy 0.512	Precison 0.512 (0.574)	Recall 0.512 (0.574)
Epoch:[3][130/424]	Loss 0.346 	Accuracy 0.590	Precison 0.590 (0.574)	Recall 0.590 (0.574)
Epoch:[3][135/424]	Loss 0.293 	Accuracy 0.660	Precison 0.660 (0.574)	Recall 0.660 (0.574)
Epoch:[3][140/424]	Loss 0.330 	Accuracy 0.668	Precison 0.668 (0.574)	Recall 0.668 (0.574)
Epoch:[3][145/424]	Loss 0.353 	Accuracy 0.520	Precison 0.520 (0.574)	Recall 0.520 (0.574)
Epoch:[3][150/424]	Loss 0.349 	Accuracy 0.516	Precison 0.516 (0.574)	Recall 0.516 (0.574)
Epoch:[3][155/424]	Loss 0.329 	Accuracy 0.633	Precison 0.633 (0.574)	Recall 0.633 (0.574)
Epoch:[3][

...

Epoch:[15][70/424]	Loss 0.322 	Accuracy 0.598	Precison 0.598 (0.594)	Recall 0.598 (0.594)
Epoch:[15][75/424]	Loss 0.300 	Accuracy 0.648	Precison 0.648 (0.594)	Recall 0.648 (0.594)
Epoch:[15][80/424]	Loss 0.371 	Accuracy 0.398	Precison 0.398 (0.594)	Recall 0.398 (0.594)
Epoch:[15][85/424]	Loss 0.339 	Accuracy 0.527	Precison 0.527 (0.594)	Recall 0.527 (0.594)
Epoch:[15][90/424]	Loss 0.373 	Accuracy 0.602	Precison 0.602 (0.594)	Recall 0.602 (0.594)
Epoch:[15][95/424]	Loss 0.319 	Accuracy 0.668	Precison 0.668 (0.594)	Recall 0.668 (0.594)
Epoch:[15][100/424]	Loss 0.343 	Accuracy 0.605	Precison 0.605 (0.594)	Recall 0.605 (0.594)
Epoch:[15][105/424]	Loss 0.345 	Accuracy 0.641	Precison 0.641 (0.594)	Recall 0.641 (0.594)
Epoch:[15][110/424]	Loss 0.243 	Accuracy 0.758	Precison 0.758 (0.594)	Recall 0.758 (0.594)
Epoch:[15][115/424]	Loss 0.297 	Accuracy 0.664	Precison 0.664 (0.594)	Recall 0.664 (0.594)
Epoch:[15][120/424]	Loss 0.315 	Accuracy 0.703	Precison 0.703 (0.594)	Recall 0.703 (0.594)
Epoch

Epoch:[16][100/424]	Loss 0.333 	Accuracy 0.668	Precison 0.668 (0.596)	Recall 0.668 (0.596)
Epoch:[16][105/424]	Loss 0.335 	Accuracy 0.652	Precison 0.652 (0.596)	Recall 0.652 (0.596)
Epoch:[16][110/424]	Loss 0.239 	Accuracy 0.707	Precison 0.707 (0.596)	Recall 0.707 (0.596)
Epoch:[16][115/424]	Loss 0.299 	Accuracy 0.656	Precison 0.656 (0.596)	Recall 0.656 (0.596)
Epoch:[16][120/424]	Loss 0.312 	Accuracy 0.727	Precison 0.727 (0.596)	Recall 0.727 (0.596)
Epoch:[16][125/424]	Loss 0.312 	Accuracy 0.781	Precison 0.781 (0.596)	Recall 0.781 (0.596)
Epoch:[16][130/424]	Loss 0.325 	Accuracy 0.656	Precison 0.656 (0.596)	Recall 0.656 (0.596)
Epoch:[16][135/424]	Loss 0.284 	Accuracy 0.680	Precison 0.680 (0.596)	Recall 0.680 (0.596)
Epoch:[16][140/424]	Loss 0.317 	Accuracy 0.711	Precison 0.711 (0.596)	Recall 0.711 (0.596)
Epoch:[16][145/424]	Loss 0.331 	Accuracy 0.562	Precison 0.562 (0.596)	Recall 0.562 (0.596)
Epoch:[16][150/424]	Loss 0.334 	Accuracy 0.516	Precison 0.516 (0.596)	Recall 0.516 (0.596)

Epoch:[17][130/424]	Loss 0.318 	Accuracy 0.691	Precison 0.691 (0.598)	Recall 0.691 (0.598)
Epoch:[17][135/424]	Loss 0.294 	Accuracy 0.660	Precison 0.660 (0.598)	Recall 0.660 (0.598)
Epoch:[17][140/424]	Loss 0.321 	Accuracy 0.707	Precison 0.707 (0.598)	Recall 0.707 (0.598)
Epoch:[17][145/424]	Loss 0.322 	Accuracy 0.574	Precison 0.574 (0.598)	Recall 0.574 (0.598)
Epoch:[17][150/424]	Loss 0.353 	Accuracy 0.523	Precison 0.523 (0.598)	Recall 0.523 (0.598)
Epoch:[17][155/424]	Loss 0.307 	Accuracy 0.594	Precison 0.594 (0.598)	Recall 0.594 (0.598)
Epoch:[17][160/424]	Loss 0.311 	Accuracy 0.586	Precison 0.586 (0.598)	Recall 0.586 (0.598)
Epoch:[17][165/424]	Loss 0.340 	Accuracy 0.660	Precison 0.660 (0.598)	Recall 0.660 (0.598)
Epoch:[17][170/424]	Loss 0.477 	Accuracy 0.000	Precison 0.000 (0.598)	Recall 0.000 (0.598)
Epoch:[17][175/424]	Loss 0.368 	Accuracy 0.547	Precison 0.547 (0.598)	Recall 0.547 (0.598)
Epoch:[17][180/424]	Loss 0.246 	Accuracy 0.750	Precison 0.750 (0.598)	Recall 0.750 (0.598)

Epoch:[18][160/424]	Loss 0.316 	Accuracy 0.676	Precison 0.676 (0.599)	Recall 0.676 (0.599)
Epoch:[18][165/424]	Loss 0.345 	Accuracy 0.660	Precison 0.660 (0.599)	Recall 0.660 (0.599)
Epoch:[18][170/424]	Loss 0.471 	Accuracy 0.000	Precison 0.000 (0.599)	Recall 0.000 (0.599)
Epoch:[18][175/424]	Loss 0.360 	Accuracy 0.617	Precison 0.617 (0.599)	Recall 0.617 (0.599)
Epoch:[18][180/424]	Loss 0.201 	Accuracy 0.758	Precison 0.758 (0.599)	Recall 0.758 (0.599)
Epoch:[18][185/424]	Loss 0.311 	Accuracy 0.617	Precison 0.617 (0.599)	Recall 0.617 (0.599)
Epoch:[18][190/424]	Loss 0.352 	Accuracy 0.457	Precison 0.457 (0.599)	Recall 0.457 (0.599)
Epoch:[18][195/424]	Loss 0.396 	Accuracy 0.832	Precison 0.832 (0.599)	Recall 0.832 (0.599)
Epoch:[18][200/424]	Loss 0.359 	Accuracy 0.539	Precison 0.539 (0.599)	Recall 0.539 (0.599)
Epoch:[18][205/424]	Loss 0.292 	Accuracy 0.711	Precison 0.711 (0.599)	Recall 0.711 (0.599)
Epoch:[18][210/424]	Loss 0.346 	Accuracy 0.539	Precison 0.539 (0.599)	Recall 0.539 (0.599)

Epoch:[19][190/424]	Loss 0.357 	Accuracy 0.469	Precison 0.469 (0.601)	Recall 0.469 (0.601)
Epoch:[19][195/424]	Loss 0.396 	Accuracy 0.832	Precison 0.832 (0.601)	Recall 0.832 (0.601)
Epoch:[19][200/424]	Loss 0.348 	Accuracy 0.578	Precison 0.578 (0.601)	Recall 0.578 (0.601)
Epoch:[19][205/424]	Loss 0.297 	Accuracy 0.699	Precison 0.699 (0.601)	Recall 0.699 (0.601)
Epoch:[19][210/424]	Loss 0.336 	Accuracy 0.555	Precison 0.555 (0.601)	Recall 0.555 (0.601)
Epoch:[19][215/424]	Loss 0.304 	Accuracy 0.613	Precison 0.613 (0.601)	Recall 0.613 (0.601)
Epoch:[19][220/424]	Loss 0.322 	Accuracy 0.723	Precison 0.723 (0.601)	Recall 0.723 (0.601)
Epoch:[19][225/424]	Loss 0.266 	Accuracy 0.656	Precison 0.656 (0.601)	Recall 0.656 (0.601)
Epoch:[19][230/424]	Loss 0.303 	Accuracy 0.621	Precison 0.621 (0.601)	Recall 0.621 (0.601)
Epoch:[19][235/424]	Loss 0.297 	Accuracy 0.594	Precison 0.594 (0.601)	Recall 0.594 (0.601)
Epoch:[19][240/424]	Loss 0.396 	Accuracy 0.660	Precison 0.660 (0.601)	Recall 0.660 (0.601)

In [ ]:
data.shape

In [ ]:
data = []
data.extend(json.load(open('Log/ImitationLog/2021-03-11_23-01-35_1.json'))['logs'])


In [ ]:
decode_state(data[0],isImitation=True)[:-1]

In [ ]:
data

In [ ]:
len([0 for i in range(69)])

In [ ]:
f = open('Log/2021-03-11_23-01-35_1.json')
data = json.load(f)

In [ ]:
data = data['logs']

In [ ]:
IMITATION_LOG_DIR = 'Log/'

In [ ]:
a = list()
for i in os.listdir(IMITATION_LOG_DIR):
    f = open(IMITATION_LOG_DIR + i)
    try:
        a.extend(json.load(f)['logs'])
    except json.JSONDecodeError:
        f.close()
        continue
    f.close()

In [ ]:
# Saving checkpoint ##########
########## Saving checkpoint ##########
score:  -8.5  average score -3.4 best score 0.17 epsilon 0.69 steps 728
/ACTION 999,
tensor(0.0756, device='cuda:0', grad_fn=<MseLossBackward>)
########## Saving checkpoint ##########
########## Saving checkpoint ##########
score:  0.5  average score -3.4 best score 0.17 epsilon 0.69 steps 729
/ACTION 999,
tensor(112.1866, device='cuda:0', grad_fn=<MseLossBackward>)
########## Saving checkpoint ##########
########## Saving checkpoint ##########
score:  -0.5  average score -3.3 best score 0.17 epsilon 0.69 steps 730
/ACTION 999,
tensor(12.4178, device='cuda:0', grad_fn=<MseLossBackward>)
########## Saving checkpoint ##########
########## Saving checkpoint ##########
score:  -1.5  average score -3.3 best score 0.17 epsilon 0.69 steps 731
/ACTION 999,
tensor(0.5178, device='cuda:0', grad_fn=<MseLossBackward>)
########## Saving checkpoint ##########
########## Saving checkpoint ##########
score:  0.5  average score -3.3 best score 0.17 epsilon 0.69 steps 732
/ACTION 999,
tensor(0.0662, device='cuda:0', grad_fn=<MseLossBackward>)
########## Saving checkpoint ##########

In [ ]:
player_life = [str_env['player_life']]
player_gold = [str_env['player_life']] 
opponent_life = [str_env['opponent_life']]
time = [str_env['time']]
player_board_card_info = list()
opponent_board_card_info = list()

for x in str_env['player_board_card_info']:
    player_board_card_info.append(x['id'])
    player_board_card_info.append(x['status'])
    player_board_card_info.append(x['damage'])
    player_board_card_info.append(x['life'])

for x in str_env['opponent_board_card_info']:
    opponent_board_card_info.append(x['id'])
    opponent_board_card_info.append(x['status'])
    opponent_board_card_info.append(x['damage'])
    opponent_board_card_info.append(x['life'])
    
player_hand_card_id = str_env['player_hand_card_id']

In [ ]:
state = player_life + player_gold + opponent_life + time + player_board_card_info + opponent_board_card_info + player_hand_card_id

In [ ]:
len(state)

In [ ]:
a = torch.Tensor([1,2,2,2])
b = torch.Tensor([1,3,55,3])

In [ ]:
torch.add(a,b)

In [ ]:
test == None

In [ ]:
np.random.choice(1)

In [ ]:
info = pd.read_csv('CardIDList.csv')

In [ ]:
card_info = info[info['ID']==2]

In [ ]:
card_info

In [ ]:
info.at[56,"Name"]